<a href="https://colab.research.google.com/github/Vignesh424/Personality-Prediction/blob/main/Personality_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import polars as pl
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pl.read_csv('/content/personality_dataset.csv')

In [ ]:
data.null_count()

Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
u32,u32,u32,u32,u32,u32,u32,u32
63,73,62,66,52,77,65,0


In [ ]:
data.head()

Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
f64,str,f64,f64,str,f64,f64,str
4.0,"""No""",4.0,6.0,"""No""",13.0,5.0,"""Extrovert"""
9.0,"""Yes""",0.0,0.0,"""Yes""",0.0,3.0,"""Introvert"""
9.0,"""Yes""",1.0,2.0,"""Yes""",5.0,2.0,"""Introvert"""
0.0,"""No""",6.0,7.0,"""No""",14.0,8.0,"""Extrovert"""
3.0,"""No""",9.0,4.0,"""No""",8.0,5.0,"""Extrovert"""


In [ ]:
#Fill Nulls
cleaned = data.with_columns(
    pl.col('Time_spent_Alone').interpolate(method='linear'),
    pl.col('Stage_fear').fill_null("Unknown"),
    pl.col('Social_event_attendance').interpolate(method='linear'),
    pl.col('Going_outside').interpolate(method='linear'),
    pl.col('Drained_after_socializing').fill_null("Unknown"),
    pl.col('Post_frequency').interpolate(method='linear')
)

In [ ]:
cleaned.head()

Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
f64,str,f64,f64,str,f64,f64,str
4.0,"""No""",4.0,6.0,"""No""",13.0,5.0,"""Extrovert"""
9.0,"""Yes""",0.0,0.0,"""Yes""",0.0,3.0,"""Introvert"""
9.0,"""Yes""",1.0,2.0,"""Yes""",5.0,2.0,"""Introvert"""
0.0,"""No""",6.0,7.0,"""No""",14.0,8.0,"""Extrovert"""
3.0,"""No""",9.0,4.0,"""No""",8.0,5.0,"""Extrovert"""


In [ ]:
le = LabelEncoder()
# Apply Label Encoding using with_columns
new_cleaned = cleaned.with_columns(
    pl.Series(le.fit_transform(cleaned['Stage_fear'])).alias('Stage_fear').cast(pl.Int64),
    pl.Series(le.fit_transform(cleaned['Drained_after_socializing'])).alias('Drained_after_socializing').cast(pl.Int64)
)

In [ ]:
new_cleaned.head()

Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
f64,i64,f64,f64,i64,f64,f64,str
4.0,0,4.0,6.0,0,13.0,5.0,"""Extrovert"""
9.0,2,0.0,0.0,2,0.0,3.0,"""Introvert"""
9.0,2,1.0,2.0,2,5.0,2.0,"""Introvert"""
0.0,0,6.0,7.0,0,14.0,8.0,"""Extrovert"""
3.0,0,9.0,4.0,0,8.0,5.0,"""Extrovert"""


In [ ]:
X = new_cleaned.drop('Personality')
Y = new_cleaned['Personality']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=50)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X,Y)

DecisionTreeClassifier()

In [ ]:
Y_Pred = dt.predict(X_test)

In [ ]:
print('Classification Report', classification_report(Y_test, Y_Pred))

Classification Report               precision    recall  f1-score   support

   Extrovert       0.97      0.99      0.98       369
   Introvert       0.99      0.96      0.97       356

    accuracy                           0.98       725
   macro avg       0.98      0.97      0.98       725
weighted avg       0.98      0.98      0.98       725



In [ ]:

#pickle file
model = open('PP2.pkl', 'wb')
pickle.dump(dt, model)

